In [749]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from statistics import mode
pd.set_option('display.max_rows', 200)

In [750]:
#downloading initial file.
df_first=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221008.txt',sep=',')
# second one to fill the empty rows after diff()
df_m1=pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_221001.txt',sep=',')


In [751]:
#concatenating files for the initail and -1 week to get values for the first row after diff()
df=pd.concat([df_m1,df_first],axis=0,ignore_index=True)
df.reset_index(inplace=True,drop=True)

In [752]:
df.isnull().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

No null values

In [753]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422384 entries, 0 to 422383
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count   Dtype 
---  ------                                                                --------------   ----- 
 0   C/A                                                                   422384 non-null  object
 1   UNIT                                                                  422384 non-null  object
 2   SCP                                                                   422384 non-null  object
 3   STATION                                                               422384 non-null  object
 4   LINENAME                                                              422384 non-null  object
 5   DIVISION                                                              422384 non-null  object
 6   DATE                                                                  422384 non-null  objec

All columns except DATE and TIME are in the correct format. Let's unite that columns and change to datetime.

In [754]:
# snake_case column names
df.columns=df.columns.str.lower().str.replace(" ","")
# converting date to datetime format
df["datetime"]=df["date"]+" "+df["time"]
df.datetime=pd.to_datetime(df["datetime"])
df.drop(columns=["date","time"],inplace=True)




In [755]:
df.head(2)

,c/a,unit,scp,station,linename,division,desc,entries,exits,datetime
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767632,2742700,2022-09-24 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,7767638,2742703,2022-09-24 04:00:00


Now features are fine.

----

This code creates multindex that will help group dataframe and calculate absolute values for entries and exits instead of relevant.

In [756]:
df['stat_id']=df["station"]+" "+df["scp"]+" "+df["c/a"]
df=df.set_index(["stat_id","datetime"]).sort_index()

df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  
stat_id                 datetime                                 
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318  
                        2022-09-24 04:00:00  15841308  19006484  
                        2022-09-24 08:00:00  15841319  19006685  
                        2022-09-24 12:00:00  15841363  19007161  
                        2022-09-24 16:00:00  15841455  19007876  
...                                               ...       ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289  
                        2022-10-07 09:00:00        39       289  
                        2022-10-07 13:00:00        39       289  
                        2022-10-07 17:00:00        39       289  
                        2022-10-07 21:00:00        39       289  

[422384 rows x 9 columns]

Some of the turnstile detectors were replaced/reseted. We can calculate that difference, but it will take a lot of time and since incorrect data is less than 0.5 percent of the dataset, more efficient solution is just drop it.

In [757]:
df.loc["125 ST 00-06-00 R258"].iloc[50:]


,c/a,unit,scp,station,linename,division,desc,entries,exits
datetime,,,,,,,,,
2022-10-04 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275772980,324331921
2022-10-04 13:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773000,324332194
2022-10-05 05:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773227,324332636
2022-10-05 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773365,324332903
2022-10-06 13:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773530,324333223
2022-10-06 21:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1325,254
2022-10-07 01:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1362,327
2022-10-07 05:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1375,359
2022-10-07 09:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1551,783


In [758]:
df.loc["WOODHAVEN BLVD 00-03-03 N329"].head()

,c/a,unit,scp,station,linename,division,desc,entries,exits
datetime,,,,,,,,,
2022-09-24 01:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662066,250875
2022-09-24 05:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662063,250887
2022-09-24 09:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16662025,250904
2022-09-24 13:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16661961,250951
2022-09-24 17:00:00,N329,R201,00-03-03,WOODHAVEN BLVD,MR,IND,REGULAR,16661870,251048


Some of the data from turnstiles is reversed in the dataset. Let's add reverse for such cases in the code below. This method gets grouped dataframe and returns absolute values for entries and exits instead of relevant.

In [759]:
res=pd.DataFrame()
def calc(temp):
    #if data if reversed - changes the order and calculates diff()
    if temp['entries'].diff().sum()<0:
        ds = pd.Series(temp['entries'].values[::-1], temp['entries'].index)       
        temp["entries_abs"]=ds.diff()
    else:
        temp['entries_abs']=temp['entries'].diff()
        
    #if data if reversed - changes the order and calculates diff()
    if temp['exits'].diff().sum()<0:
        ds = pd.Series(temp['exits'].values[::-1], temp['exits'].index)       
        temp["exits_abs"]=ds.diff()
    else:
        temp['exits_abs']=temp['exits'].diff()
    return temp

In [760]:
df=df.groupby(level=0).apply(lambda x: calc(x))


In [761]:
df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  entries_abs  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318          NaN   
                        2022-09-24 04:00:00  15841308  19006484         21.0   
                        2022-09-24 08:00:00  15841319  19006685         11.0   
                        2022-09-24 12:00:00  15841363  19007161         44.0   
                        2022-09-24 16:00:00  15841455  19007876         92.0   
...                                               ...       ...          ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289          0.0   
                        2022-10-07 09:00:00        39       289          0.0   
                        2022-10-07 13:00:00        39       289          0.0   
                        2022-10-07 17:00:00        39       289          0.0   
                        2022-10-07 21:00:00        39       289          0.0   

                                             exits_abs  
stat_id                 datetime                        
1 AV 00-00-00 H007      2022-09-24 00:00:00        NaN  
                        2022-09-24 04:00:00      166.0  
                        2022-09-24 08:00:00      201.0  
                        2022-09-24 12:00:00      476.0  
                        2022-09-24 16:00:00      715.0  
...                                                ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        0.0  
                        2022-10-07 09:00:00        0.0  
                        2022-10-07 13:00:00        0.0  
                        2022-10-07 17:00:00        0.0  
                        2022-10-07 21:00:00        0.0  

[422384 rows x 11 columns]

Dropping turnstiles with negative values in diff()

In [762]:
df=df.drop(df.index[df['entries_abs'] < 0])
df=df.drop(df.index[df['exits_abs'] < 0])


In [763]:
df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 04:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 08:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 12:00:00  H007  R248  00-00-00       1 AV   
                        2022-09-24 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 09:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 13:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 17:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-07 21:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2022-09-24 00:00:00        L      BMT  REGULAR   
                        2022-09-24 04:00:00        L      BMT  REGULAR   
                        2022-09-24 08:00:00        L      BMT  REGULAR   
                        2022-09-24 12:00:00        L      BMT  REGULAR   
                        2022-09-24 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        6      IRT  REGULAR   
                        2022-10-07 09:00:00        6      IRT  REGULAR   
                        2022-10-07 13:00:00        6      IRT  REGULAR   
                        2022-10-07 17:00:00        6      IRT  REGULAR   
                        2022-10-07 21:00:00        6      IRT  REGULAR   

                                              entries     exits  entries_abs  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2022-09-24 00:00:00  15841287  19006318          NaN   
                        2022-09-24 04:00:00  15841308  19006484         21.0   
                        2022-09-24 08:00:00  15841319  19006685         11.0   
                        2022-09-24 12:00:00  15841363  19007161         44.0   
                        2022-09-24 16:00:00  15841455  19007876         92.0   
...                                               ...       ...          ...   
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        39       289          0.0   
                        2022-10-07 09:00:00        39       289          0.0   
                        2022-10-07 13:00:00        39       289          0.0   
                        2022-10-07 17:00:00        39       289          0.0   
                        2022-10-07 21:00:00        39       289          0.0   

                                             exits_abs  
stat_id                 datetime                        
1 AV 00-00-00 H007      2022-09-24 00:00:00        NaN  
                        2022-09-24 04:00:00      166.0  
                        2022-09-24 08:00:00      201.0  
                        2022-09-24 12:00:00      476.0  
                        2022-09-24 16:00:00      715.0  
...                                                ...  
ZEREGA AV 00-05-01 R419 2022-10-07 05:00:00        0.0  
                        2022-10-07 09:00:00        0.0  
                        2022-10-07 13:00:00        0.0  
                        2022-10-07 17:00:00        0.0  
                        2022-10-07 21:00:00        0.0  

[420431 rows x 11 columns]

In [764]:
df[df["entries_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


In [765]:
df[df["exits_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


Dropping all NaN values and changing size of dataset to one week.

In [766]:
df.isnull().sum()

c/a               0
unit              0
scp               0
station           0
linename          0
division          0
desc              0
entries           0
exits             0
entries_abs    5041
exits_abs      5041
dtype: int64

In [767]:
df.dropna(inplace=True)

In [768]:
df.isnull().sum()

c/a            0
unit           0
scp            0
station        0
linename       0
division       0
desc           0
entries        0
exits          0
entries_abs    0
exits_abs      0
dtype: int64

In [769]:
df=df.reset_index(level=0).loc["2022-10-01":"2022-10-07"]

In [770]:
df

,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,,
2022-10-01 00:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845079,19027035,92.0,474.0
2022-10-01 04:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845098,19027234,19.0,199.0
2022-10-01 08:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845108,19027401,10.0,167.0
2022-10-01 12:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845163,19027781,55.0,380.0
2022-10-01 16:00:00,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845254,19028361,91.0,580.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0
2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0
2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R419,R326,00-05-01,ZEREGA AV,6,IRT,REGULAR,39,289,0.0,0.0


----

There is another problem. As you can see on the "entries" column, for the turnstile 01-05-00 at WORLD TRADE CTR Station, calculated number of entries was 2140135519, and in eight hours it changes to 8098. Probably turnstile was reseted or replased for some reasons. But that seriously affects absolute value calculation. I'll replace those values with mean for that station on the other day. The threshold to detect such fluctuations will be if that absolute value is greater than 15000. I follow the logic that it is phisically impossible to pass turnstile faster than for a second. For a period of 4 hours this gives us theoretical max amount of people at 4 * 60 * 60 = 14400.

In [771]:
df[df["station"]=='WORLD TRADE CTR'].iloc[527:530]

,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,,
2022-10-01 00:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,REGULAR,2140135519,9,2.140127e+09,0.0
2022-10-01 08:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,RECOVR AUD,8098,9,1.000000e+00,0.0
2022-10-01 12:00:00,WORLD TRADE CTR 01-05-00 N094,N094,R029,01-05-00,WORLD TRADE CTR,ACE23,IND,RECOVR AUD,8099,9,1.000000e+00,0.0


I'll drop the index to find and replace these incorrect values with mean of their station respectively.

In [773]:
df.reset_index(inplace=True)

In [774]:
for x,k in df.iterrows():
    #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
    if df.loc[x,"entries_abs"]>15000:
        mean=df[(df["entries_abs"]<15000) & (df["station"]==k[5])]["entries_abs"].mean()
        mean=int(mean)    
        df.loc[x,"entries_abs"]=mean

114.19622641509434 10534
70.8645707376058 17000
82.92361883574341 75024
112.87530562347189 108410
115.68731848983543 155344
115.68731848983542 155350
68.19200695047785 161473
66.11011904761905 187644
87.46364379084967 196933
33.28620019436346 202791
47.330798479087456 207939


In [775]:
for x,k in df.iterrows():
    #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
    if df.loc[x,"exits_abs"]>15000:
        mean=df[(df["exits_abs"]<15000) & (df["station"]==k[5])]["exits_abs"].mean()
        mean=int(mean)    
        df.loc[x,"exits_abs"]=mean

No incorrect data.

In [776]:
df[df["entries_abs"]>15000]

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


In [778]:
df[df["exits_abs"]>15000]

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


Changing entries_abs and exits_abs to int

In [780]:
df["entries_abs"]=df["entries_abs"].astype(int)
df["exits_abs"]=df["exits_abs"].astype(int)

In [781]:
df.head(1)

,datetime,stat_id,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
0,2022-10-01,1 AV 00-00-00 H007,H007,R248,00-00-00,1 AV,L,BMT,REGULAR,15845079,19027035,92,474


Now, let's drop: 
- entries and exits (we already used them)
- division - this feature contains information about station owner. This won't be useful for this project.
- c/a - this feature contains information about electrical station that supplies energy for current station. This won't be useful for this project either.
- desc - represent the "REGULAR" scheduled audit event. This won't be useful for this project.


In [782]:
df.drop(columns=["entries","exits","c/a","division","desc"],inplace=True)
df

,datetime,stat_id,unit,scp,station,linename,entries_abs,exits_abs
0,2022-10-01 00:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,92,474
1,2022-10-01 04:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,19,199
2,2022-10-01 08:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,10,167
3,2022-10-01 12:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,55,380
4,2022-10-01 16:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,91,580
...,...,...,...,...,...,...,...,...
210010,2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210011,2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210012,2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
210013,2022-10-07 17:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0


Saving file to csv.

In [785]:
df.to_csv('../data/df_cleaned.csv',index=False)

In [786]:
dd=pd.read_csv('../data/df_cleaned.csv',index_col="datetime")
dd

,stat_id,unit,scp,station,linename,entries_abs,exits_abs
datetime,,,,,,,
2022-10-01 00:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,92,474
2022-10-01 04:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,19,199
2022-10-01 08:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,10,167
2022-10-01 12:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,55,380
2022-10-01 16:00:00,1 AV 00-00-00 H007,R248,00-00-00,1 AV,L,91,580
...,...,...,...,...,...,...,...
2022-10-07 05:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
2022-10-07 09:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
2022-10-07 13:00:00,ZEREGA AV 00-05-01 R419,R326,00-05-01,ZEREGA AV,6,0,0
